In [ ]:
from pathlib import Path
from astropy import units as u
from astropy.stats import sigma_clipped_stats
from astropy.table import Table
from astropy.time import Time
from astropy.timeseries import aggregate_downsample
from astropy.timeseries import TimeSeries
from astropy.timeseries import BinnedTimeSeries

import re
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def boundify(a, sigma=2.0):
    mean = a.mean()
    dev = np.std(a)
    
    return (mean - (sigma * dev)).value, (mean + (sigma * dev)).value

def filter_nan(ts, filter_column):
    nans = np.isnan(ts[filter_column].data)
    return ts[~nans]

def filter_inf(ts, filter_column):
    infs = np.isinf(ts[filter_column].data)
    return ts[~infs]

def check_filter(labels):
    values = []
    
    for label in labels:
        if re.match(r'.*_B[0-9]+_.*', str(label)):
            values.append('Blue')
        elif re.match(r'.*_V[0-9]+_.*', str(label)):
            values.append('Green')
        elif re.match(r'.*_R[0-9]+_.*', str(label)):
            values.append('Red')
        else:
            values.append('Unknown')
    
    return values

def load_csv(path):
    t = Table.read(path, format='ascii.csv', delimiter=',', fast_reader=False)
    
    t['JD_UTC'].unit = u.day
    t['time'] = Time(t['JD_UTC'], format='jd')
    t['filter'] = check_filter(t['Label'])

    # remove unwanted data
    t.keep_columns(['time','rel_flux_T1', 'rel_flux_err_T1', 'Source_AMag_T1', 'Source_AMag_Err_T1', 'Source_SNR_T1', 'filter'])
    t['rel_flux_T1'].unit = 'electron/s'
    t['rel_flux_err_T1'].unit = 'electron/s'
    t['Source_AMag_T1'].unit = 'mag'
    t['Source_AMag_Err_T1'].unit = 'mag'
    
    return t
    
    
plot_value = 'Source_AMag_T1'
plot_sigma = 'Source_AMag_Err_T1'

In [ ]:
# constants
object_name = 'CRTS J144508.6+050514'
working_directory = Path('/data/PIRATE/FITS/') / object_name
csv_data = working_directory / 'measurements.csv'

# TODO: automatically infer this from the FITS data
period = 10.903857574156442 * u.hour

plt.style.use('guide.mplstyle')

In [ ]:
ts_red = TimeSeries(load_csv(working_directory / 'measurements-red.csv'))
ts_green = TimeSeries(load_csv(working_directory / 'measurements-green.csv'))
ts_blue = TimeSeries(load_csv(working_directory / 'measurements-blue.csv'))

In [ ]:
ts_red_folded = ts_red.fold(period=period, normalize_phase=False, wrap_phase=period, epoch_phase=0.2 * period)
ts_green_folded = ts_green.fold(period=period, normalize_phase=False, wrap_phase=period, epoch_phase=0.2 * period)
ts_blue_folded = ts_blue.fold(period=period, normalize_phase=False, wrap_phase=period, epoch_phase=0.2 * period)

print(period.to(u.day))

In [ ]:
fig = plt.figure(figsize=(20,20))

xlabel = 'Phase'
ylabel = 'Magnitude'
kwargs = { 'markersize': 3, 'marker': '^', 'linestyle': '' }

y_min, y_max = boundify(ts_red_folded[plot_value], sigma=3)
ax1 = fig.add_subplot(411)
#ax1.plot(ts_red.time, ts_red[plot_value].value, 'r.', **kwargs)
#ax1.plot(ts_red.time - 1.0, ts_red[plot_value].value, 'r.', **kwargs)
ax1.plot((ts_red_folded.time.jd / period.to(u.day)).value, ts_red_folded[plot_value].value, 'r.', **kwargs)
ax1.plot((ts_red_folded.time.jd / period.to(u.day)).value - 1.0, ts_red_folded[plot_value].value, 'r.', **kwargs)
#ax1.plot((ts_red_mean.time_bin_start.jd / period.to(u.day)).value, ts_red_mean[plot_value].value, 'k.', **kwargs)
#ax1.plot((ts_red_mean.time_bin_start.jd / period.to(u.day)).value - 1.0, ts_red_mean[plot_value].value, 'k.', **kwargs)
ax1.set_ylim(y_min, y_max)
ax1.set_xlabel(xlabel)
ax1.set_ylabel(ylabel)
ax1.invert_yaxis()
ax1.grid()

y_min, y_max = boundify(ts_green_folded[plot_value], sigma=3)
ax2 = fig.add_subplot(412)
#ax2.plot(ts_green.time, ts_green[plot_value].value, 'g.', **kwargs)
#ax2.plot(ts_green.time - 1.0, ts_green[plot_value].value, 'g.', **kwargs)
ax2.plot((ts_green_folded.time.jd / period.to(u.day)).value, ts_green_folded[plot_value].value, 'g.', **kwargs)
ax2.plot((ts_green_folded.time.jd / period.to(u.day)).value - 1.0, ts_green_folded[plot_value].value, 'g.', **kwargs)
#ax2.plot((ts_green_mean.time_bin_start.jd / period.to(u.day)).value, ts_green_mean[plot_value].value, 'k.', **kwargs)
#ax2.plot((ts_green_mean.time_bin_start.jd / period.to(u.day)).value - 1.0, ts_green_mean[plot_value].value, 'k.', **kwargs)
ax2.set_ylim(y_min, y_max)
ax2.set_xlabel(xlabel)
ax2.set_ylabel(ylabel)
ax2.invert_yaxis()
ax2.grid()

#y_min, y_max = boundify(ts_blue_folded[plot_value], sigma=1)
ax3 = fig.add_subplot(413)
#ax3.plot(ts_blue.time, ts_blue[plot_value].value, 'b.', **kwargs)
#ax3.plot(ts_blue.time - 1.0, ts_blue[plot_value].value, 'b.', **kwargs)
ax3.plot((ts_blue_folded.time.jd / period.to(u.day)).value, ts_blue_folded[plot_value].value, 'b.', **kwargs)
ax3.plot((ts_blue_folded.time.jd / period.to(u.day)).value - 1.0, ts_blue_folded[plot_value].value, 'b.', **kwargs)
#ax3.plot((ts_blue_mean.time_bin_start.jd / period.to(u.day)).value, ts_blue_mean[plot_value].value, 'k.', **kwargs)
#ax3.plot((ts_blue_mean.time_bin_start.jd / period.to(u.day)).value - 1.0, ts_blue_mean[plot_value].value, 'k.', **kwargs)
ax3.set_ylim(y_min, y_max)
ax3.set_xlabel(xlabel)
ax3.set_ylabel(ylabel)
ax3.invert_yaxis()
ax3.grid()

y_min, y_max = boundify(ts_green_folded[plot_value], sigma=3)
ax4 = fig.add_subplot(414)
#ax4.plot(ts_red.time, ts_red[plot_value].value, 'r.', **kwargs)
#ax4.plot(ts_red.time - 1.0, ts_red[plot_value].value, 'r.', **kwargs)
#ax4.plot(ts_green.time, ts_green[plot_value].value, 'g.', **kwargs)
#ax4.plot(ts_green.time - 1.0, ts_green[plot_value].value, 'g.', **kwargs)
#ax4.plot(ts_blue.time, ts_blue[plot_value].value, 'b.', **kwargs)
#ax4.plot(ts_blue.time - 1.0, ts_blue[plot_value].value, 'b.', **kwargs)
ax4.plot((ts_red_folded.time.jd / period.to(u.day)).value, ts_red_folded[plot_value].value, 'r.', **kwargs)
ax4.plot((ts_red_folded.time.jd / period.to(u.day)).value - 1.0, ts_red_folded[plot_value].value, 'r.', **kwargs)
ax4.plot((ts_green_folded.time.jd / period.to(u.day)).value, ts_green_folded[plot_value].value, 'g.', **kwargs)
ax4.plot((ts_green_folded.time.jd / period.to(u.day)).value - 1.0, ts_green_folded[plot_value].value, 'g.', **kwargs)
ax4.plot((ts_blue_folded.time.jd / period.to(u.day)).value, ts_blue_folded[plot_value].value, 'b.', **kwargs)
ax4.plot((ts_blue_folded.time.jd / period.to(u.day)).value - 1.0, ts_blue_folded[plot_value].value, 'b.', **kwargs)
ax4.set_ylim(y_min, y_max)
ax4.set_xlabel(xlabel)
ax4.set_ylabel(ylabel)
ax4.invert_yaxis()
ax4.grid()

fig.show()

In [ ]:
fig.savefig(working_directory / (object_name + '.png'), format='png')

In [ ]:
r_mag = ts_red[plot_value].mean()
v_mag = ts_green[plot_value].mean()
b_mag = ts_blue[plot_value].mean()
bv_mag = 0.3

print('R = ', r_mag)
print('V = ', v_mag)
print('B = ', b_mag)
print('B-V = ', bv_mag)
print('V-R = ', v_mag - r_mag)

print('Teff = ', np.power(10, ((bv_mag - 14.551) / -3.684)))
print('Teff = ', 4600.0 * ((1.0/((0.92 * (bv_mag)) + 1.7)) + (1.0/((0.92 * (bv_mag)) + 0.62))))

In [ ]:
nbins = 20
ts_red_binned = BinnedTimeSeries(ts_red, time_bin_start=ts_red.time[0], time_bin_end=ts_red.time[0] + period, time_bin_size=period / nbins)
ts_green_binned = BinnedTimeSeries(ts_green, time_bin_start=ts_green.time[0], time_bin_end=ts_red.time[0] + period, time_bin_size=period / nbins)
ts_blue_binned = BinnedTimeSeries(ts_blue, time_bin_start=ts_blue.time[0], time_bin_end=ts_blue.time[0] + period, time_bin_size=period / nbins)



In [ ]:
export_columns=['time', 'Source_AMag_T1', 'Source_AMag_Err_T1']

ts_red[export_columns].write(working_directory / 'red.phoebe', format='ascii.csv', delimiter=' ')
ts_green[export_columns].write(working_directory / 'green.phoebe', format='ascii.csv', delimiter=' ')
ts_blue[export_columns].write(working_directory / 'blue.phoebe', format='ascii.csv', delimiter=' ')

ts_bv = ts_red
ts_bv[plot_value] = 